In [1]:
from pyspark.sql import types as T
from pyspark.sql import functions as F 
from pyspark.sql import DataFrame
from pyspark.sql.functions import * 
from pyspark.sql.window import Window
import sys

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
463,application_1590480489834_0333,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#steps:
# Import the open, click, visit, unsubscribe and AIT for IJM, JA and WD at user level from Selligent through Sandbox (from 01-04-2020 until today)
# Group the user level data on campaign type
# visualize each group's KPIs in a comparative view

In [14]:
#email data 
#######################################################################################################################
#selligent_df includes send, open, unique_open, click, unique_click from selligent for three campaigns IJM, JA, and WD
    # date_trunc by 'week'---> change it to daily
    # add JA click
    
#test the query for 3 days:
# selligent_df.show(5)

# +----------+-------+-------------+----+----------+-----------+------------+-------------+
# |action_day|user_id|campaign_type|send|total_open|unique_open|total_clicks|unique_clicks|
# +----------+-------+-------------+----+----------+-----------+------------+-------------+
# |2020-08-25|5879114|       Others|   1|         0|          0|           0|            0|
# |2020-08-25|3635261|       Others|   1|         0|          0|           0|            0|
# |2020-08-25|7987656|       Others|   1|         0|          0|           0|            0|
# |2020-08-25|5963783|           JA|   6|         0|          0|           0|            0|
# |2020-08-25|5039172|           JA|   1|         1|          1|           0|            0|
# +----------+-------+-------------+----+----------+-----------+------------+-------------+



selligent_df = spark.sql("""
    SELECT
        to_date(concat_ws('-', cast(year as string), cast(month as string), cast(day as string))) as action_day,
        user_login_id as user_id,
        campaign_type,
        floor(sum(sent)) as send, 
        floor(sum(opened)) as total_open,
        count(distinct case when opened > 0 then ditem_id end) as unique_open,
        floor(sum(clicked)) as total_clicks,
        count(distinct case when clicked > 0 then ditem_id end) as unique_clicks
    FROM live_raw_selligent_de_stepstone.flags_enriched fe
    WHERE  
        (campaign_type in ('JA', 'IJM', 'WeeklyDigest')
        or campaign_id in (95183, 95184)) 
        and year = 2020 
        and month = 4
        and day >= 1
    GROUP BY action_day, campaign_type, user_id""").\
    union(spark.sql("""
    select
        to_date(concat_ws('-', cast(year as string), cast(month as string), cast(day as string))) as action_day,
        userlogin_id as user_id, 
        'JA' as campaign_type,
        0 as send,
        0 as total_open,
        0 as unique_open,
        count(*) as total_clicks,
        count(distinct jaid) as unique_clicks
    from stepstonede.jaclickofferid A
    right join stepstonede.userlogin_jobuser B on A.jaid = B.jobuser_id
    WHERE 
        year = 2020 
        and month = 4
        and day >= 1
    group by action_day, campaign_type, user_id
    """))

# selligent_df.show(5)
# +----------+-------+-------------+----+----------+-----------+------------+-------------+
# |action_day|user_id|campaign_type|send|total_open|unique_open|total_clicks|unique_clicks|
# +----------+-------+-------------+----+----------+-----------+------------+-------------+
# |2020-04-28|4629718|       Others|   0|         1|          1|           0|            0|
# |2020-04-28|7990265|          IJM|   5|         0|          0|           0|            0|
# |2020-04-28|8470140|          IJM|   3|         0|          0|           0|            0|
# |2020-04-28|8878126|       Others|   1|         0|          0|           0|            0|
# |2020-04-28|8336624|       Others|   1|         1|          1|           0|            0|
# +----------+-------+-------------+----+----------+-----------+------------+-------------+


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
#get unsubscribe data for url through presentation.hits_de
    # Search for recognised_user_id in the table
    # Filter on url
###############################################################################################################################

#unsubscribe data from website through presentation.hits_de, presentation.hits_android_global

#notes:
    #1.
        #Internet irregularities, system irregularities, or the use of custom visitor IDs can rarely use 
        #the same visit_num values for different visits. Use visit_start_time_gmt when counting visits to make sure that these visits are counted.
    #2.
        #Count the number of rows where a value is in post_pagename or post_page_url
        # Concatenate post_visid_high , post_visid_low , visit_num , and visit_start_time_gmt.
        # Count the unique number of values.
        
        
#1 add campaign dentifier: use post_campaign
            # case when upper(AA.post_campaign) like 'JAJOB%' then 'JA' else
            #             case when upper(AA.post_campaign) like 'IJM%' then 'IJM' else 'WeeklyDigest' end end as campaign_type  
# 2. add AIT and listing view from hits_de
# 3: take a sample of users and check if they unsubscribed from what campaigns. 
        

Website_unsub_visits_df = spark.sql("""
    select distinct 
          user_id, 
          user_hash, 
          job_agent_id, 
          to_date(visit_start_time_gmt) as visit_date
    from presentation.hits_de AA
    where (AA.p35_url like '%unsub%' or AA.p35_url like '%Unsub%')
          and exclude_hit <= 0 
          and year = 2020
          and month >= 4
          and day >= 1

    union all

    select distinct 
          user_id, 
          user_hash, 
          job_agent_id, 
          to_date(visit_start_time_gmt) as visit_date
    from presentation.hits_android_global AA
    where AA.job_site like '%-de' 
          and (AA.p35_url like '%unsub%' or AA.p35_url like '%Unsub%') 
          and exclude_hit <= 0 
          and year = 2020
          and month >= 4
          and day >= 1

    union all

    select distinct 
          user_id, 
          user_hash, 
          job_agent_id, 
          to_date(visit_start_time_gmt) as visit_date
    from presentation.hits_iphoneapp_global AA
    where AA.job_site like '%-de' 
          and (AA.p35_url like '%unsub%' or AA.p35_url like '%Unsub%')
          and exclude_hit <= 0 
          and year = 2020
          and month >= 4
          and day >= 1""")

# Website_unsub_visits_df.show(5)
# +-------+--------------------+------------+----------+
# |user_id|           user_hash|job_agent_id|visit_date|
# +-------+--------------------+------------+----------+
# |9418255|cb64310a-cb18-4ea...|        null|2020-08-27|
# |9339154|8dfa7aa7-c837-4ea...|        null|2020-08-23|
# |9378525|b80ef99f-cb7c-435...|        null|2020-08-23|
# |7410858|67692f13-4b48-4a2...|    15461012|2020-08-23|
# |4685972|e45a9f05-34d3-4cf...|    14469011|2020-08-23|
# +-------+--------------------+------------+----------+


Website_unsub_visits_df = Website_unsub_visits_df.groupBy("user_id", "visit_date"). \
    agg(F.countDistinct(F.col("user_id")).alias("recipient_unsub"))


#agg(F.countDistinct(F.col("user_id"))
#Website_unsub_visits_df.show(5)

# +-------+----------+---------------+
# |user_id|visit_date|recipient_unsub|
# +-------+----------+---------------+
# |3066967|2020-08-17|              1|
# |5751319|2020-08-21|              1|
# |8545543|2020-08-02|              1|
# |2224437|2020-08-09|              1|
# |8721124|2020-08-18|              1|
# +-------+----------+---------------+

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
#visit and AIT data:
#Visits
    #Concatenate post_visid_high , post_visid_low , visit_num , and visit_start_time_gmt .
    #Count the unique number of values.

Website_visits_df = spark.sql("""
    select distinct 
          user_id, 
          user_hash, 
          job_agent_id, 
          to_date(visit_start_time_gmt) as visit_date,
          visit_num
    from presentation.hits_de AA
    where exclude_hit <= 0 
          and year = 2020
          and month >= 4
          and day >= 1

    union all

    select distinct 
          user_id, 
          user_hash, 
          job_agent_id, 
          to_date(visit_start_time_gmt) as visit_date,
          visit_num
    from presentation.hits_android_global AA
    where AA.job_site like '%-de' 
          and exclude_hit <= 0 
          and year = 2020
          and month >= 4
          and day >= 1

    union all

    select distinct 
          user_id, 
          user_hash, 
          job_agent_id, 
          to_date(visit_start_time_gmt) as visit_date,
          visit_num
    from presentation.hits_iphoneapp_global AA
    where AA.job_site like '%-de' 
          and exclude_hit <= 0 
          and year = 2020
          and month >= 4
          and day >= 1""")

#Website_visits_df.show(5)
# +-------+--------------------+------------+----------+
# |user_id|           user_hash|job_agent_id|visit_date|
# +-------+--------------------+------------+----------+
# |9418255|cb64310a-cb18-4ea...|        null|2020-08-27|
# |9339154|8dfa7aa7-c837-4ea...|        null|2020-08-23|
# |9378525|b80ef99f-cb7c-435...|        null|2020-08-23|
# |7410858|67692f13-4b48-4a2...|    15461012|2020-08-23|
# |4685972|e45a9f05-34d3-4cf...|    14469011|2020-08-23|
# +-------+--------------------+------------+----------+


Website_visits_df = Website_visits_df.groupBy("user_id", "visit_num", "visit_date"). \
    agg(F.countDistinct(F.col("user_id")).alias("visit"))

 

#agg(F.countDistinct(F.col("user_id"))
# Website_visits_df.show(5)
# +-------+---------+----------+-----+
# |user_id|visit_num|visit_date|visit|
# +-------+---------+----------+-----+
# |8320707|       15|2020-08-03|    1|
# |6240646|        6|2020-06-30|    1|
# |6953249|       18|2020-08-04|    1|
# |4348184|       58|2020-07-14|    1|
# |4186578|        1|2020-07-07|    1|
# +-------+---------+----------+-----+

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+----------+-----+
|user_id|visit_num|visit_date|visit|
+-------+---------+----------+-----+
|8320707|       15|2020-08-03|    1|
|6240646|        6|2020-06-30|    1|
|6953249|       18|2020-08-04|    1|
|4348184|       58|2020-07-14|    1|
|4186578|        1|2020-07-07|    1|
+-------+---------+----------+-----+
only showing top 5 rows

In [27]:
Website_visits_unsub_df = Website_visits_df.join(Website_unsub_visits_df, "user_id", "left").distinct()

# Website_visits_unsub_df.show(5)
# +-------+----------+-----+----------+---------------+
# |user_id|visit_date|visit|visit_date|recipient_unsub|
# +-------+----------+-----+----------+---------------+
# | 100320|2020-07-21|    1|      null|           null|
# |1005313|2020-07-20|    1|2020-06-16|              1|
# |1005313|2020-06-02|    1|2020-06-16|              1|
# |1005313|2020-07-06|    1|2020-06-16|              1|
# |1005313|2020-06-09|    1|2020-06-16|              1|
# +-------+----------+-----+----------+---------------+

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
#join selligant and website traffic and groupby:

selligant_website_recipient_df = selligent_df.join(Website_visits_unsub_df, "user_id", "left").distinct().\
    groupBy("user_id", "action_day", "recipient_unsub", "visit", "visit_num", "campaign_type", "send", "unique_open", "unique_clicks"). \
    agg(F.sum(F.col("send")).alias("recipient"))

        

# selligant_website_df.show(5)
# +-------+----------+----------+-------------+---------+
# |user_id|action_day|visit_date|campaign_type|sum(send)|
# +-------+----------+----------+-------------+---------+
# |1168557|2020-04-29|2020-08-09|       Others|        0|
# |1168557|2020-04-29|2020-08-11|       Others|        0|
# |1168557|2020-04-09|2020-08-09|           JA|        1|
# |1168557|2020-04-09|2020-08-11|           JA|        1|
# |1168557|2020-04-24|2020-08-09|           JA|        1|
# +-------+----------+----------+-------------+---------+


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
selligant_website_recipient_df = selligent_df.join(Website_visits_unsub_df, "user_id", "left").distinct().\
    groupBy("user_id", "action_day", "recipient_unsub", "visit", "visit_num", "campaign_type", "send", "unique_open", "unique_clicks"). \
    agg(F.sum(F.col("unique_open")).alias("recipient_unique_open"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
selligant_website_recipient_df = selligent_df.join(Website_visits_unsub_df, "user_id", "left").distinct().\
    groupBy("user_id", "action_day", "recipient_unsub", "visit", "visit_num", "campaign_type", "send", "unique_open", "unique_clicks"). \
    agg(F.sum(F.col("unique_clicks")).alias("recipient_unique_clicks"))


selligant_website_recipient_df.show(5)
# +-------+----------+---------------+-----+-------------+----+-----------+-------------+-----------------------+
# |user_id|action_day|recipient_unsub|visit|campaign_type|send|unique_open|unique_clicks|recipient_unique_clicks|
# +-------+----------+---------------+-----+-------------+----+-----------+-------------+-----------------------+
# |1005313|2020-04-15|              1|    1|           JA|   1|          1|            0|                      0|
# |1005313|2020-04-01|              1|    1|           JA|   1|          1|            0|                      0|
# |1005313|2020-04-02|              1|    1|           JA|   1|          1|            0|                      0|
# |1005313|2020-04-11|              1|    1|           JA|   1|          1|            0|                      0|
# |1005313|2020-04-14|              1|    1|           JA|   0|          0|            1|                     13|
# +-------+----------+---------------+-----+-------------+----+-----------+-------------+-----------------------+


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
#unsubscribe per recipient for compaign groups:
selligant_website_recipient_df.crosstab("campaign_type", "recipient_unsub").show()


#without filtering on campaign type:
# +-----------------------------+-------+--------+
# |campaign_type_recipient_unsub|      1|    null|
# +-----------------------------+-------+--------+
# |                           JA|2062973|26108554|
# |                          IJM|1534735|13368049|
# |                       Others| 252582| 2995383|
# |                          CLC|  17833|  106056|
# +-----------------------------+-------+--------+


#with filtering on three campaign types
# +-----------------------------+-------+--------+
# |campaign_type_recipient_unsub|      1|    null|
# +-----------------------------+-------+--------+
# |                           JA|2062973|26108554|
# |                          IJM|1534735|13368049|
# |                       Others|  96310|  818056|
# +-----------------------------+-------+--------+

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------+--------+---------+
|campaign_type_recipient_unsub|       1|     null|
+-----------------------------+--------+---------+
|                           JA|42756796|223306954|
|                          IJM|27361828|150391758|
|                       Others| 1939857|  8259702|
+-----------------------------+--------+---------+